In [9]:
# Install a pip package in the current Jupyter kernel
import sys
!{sys.executable} -m pip install nltk tensorflow-hub

You are using pip version 10.0.1, however version 18.0 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [10]:
import preprocessor  # this will download some data

In [11]:
import preprocessor
import pandas as pd
import pickle
import os

In [12]:
def maybe_process(store_file="data.pkl", force=False):
    if force or not os.path.exists(store_file):
        data = preprocessor.process_dir("dump")
        with open(store_file, 'wb') as data_file:
            pickle.dump(data, data_file)
    else:
        with open(store_file, 'rb') as data_file:
            data = pickle.load(data_file)
    return data

In [13]:
data = maybe_process()

In [14]:
df = pd.DataFrame(data)
df = df.fillna("Unknown")

In [15]:
user_vocabulary = pd.concat([df["assignee"], df["reporter"]]).unique()
assignee_vocabulary = df["assignee"].unique()

In [16]:
with open("vocabulary.pkl", 'wb') as vocabulary_file:
    pickle.dump(user_vocabulary, vocabulary_file)
with open("assignee_vocabulary.pkl", 'wb') as assignee_assignee_vocabulary_file:
    pickle.dump(assignee_vocabulary, assignee_assignee_vocabulary_file)

In [17]:
vocabulary_map = {"Unknown": 0}
count = 0
for el in user_vocabulary:
    count += 1
    vocabulary_map[el] = count

In [18]:
with open("vocabulary.pkl", 'wb') as vocabulary_file:
    pickle.dump(user_vocabulary, vocabulary_file)

with open("vocabulary_map.pkl", 'wb') as vocabulary_file:
    pickle.dump(vocabulary_map, vocabulary_file)

Start tensorflow stuff here

In [19]:
import tensorflow as tf
import tensorflow_hub as hub
import shutil
import numpy as np

print(tf.__version__)
tf.logging.set_verbosity(tf.logging.INFO)
pd.options.display.max_rows = 10
pd.options.display.float_format = '{:.1f}'.format

1.9.0


Now, split the data into two parts -- training and evaluation.

In [20]:
np.random.seed(seed=1) #makes result reproducible
msk = np.random.rand(len(df)) < 0.8
traindf = df[msk]
evaldf = df[~msk]

In [21]:
def add_more_features(df):
    # TODO: Add more features to the dataframe
    df["summary_clean"] = df["summary"].apply(lambda x: " ".join(preprocessor.process_text(x)))
    df["description_clean"] = df["description"].apply(lambda x: " ".join(preprocessor.process_text(x)))
    return df

In [22]:
# Create pandas input function
def make_input_fn(df, num_epochs):
    return tf.estimator.inputs.pandas_input_fn(
        x = add_more_features(df),
#         y = df["assignee"].fillna('Unknown').apply(lambda x: vocabulary_map[x]),
        y = df["assignee"],
        batch_size = 128,
        num_epochs = num_epochs,
        shuffle = True,
        queue_capacity = 1000,
        num_threads = 1
      )

In [23]:
# Define your feature columns
def create_feature_cols():
    return [
#         tf.feature_column.categorical_column_with_vocabulary_list("reporter", 
#                                                                   vocabulary_list=user_vocabulary.tolist(), 
#                                                                   default_value=0)


        tf.feature_column.embedding_column(
            tf.feature_column.categorical_column_with_vocabulary_list(
                "reporter", vocabulary_list=user_vocabulary.tolist(), 
                default_value=0),
            1500,
        ),
        
#         hub.text_embedding_column(
#             key="description_clean", 
#             module_spec="https://tfhub.dev/google/nnlm-en-dim128/1",
#             trainable=True,
#         ),
        
#         hub.text_embedding_column(
#             key="summary_clean", 
#             module_spec="https://tfhub.dev/google/nnlm-en-dim128/1",
#             trainable=True,
#         ),
        
        hub.text_embedding_column(
            key="description_clean", 
            module_spec="https://tfhub.dev/google/Wiki-words-250-with-normalization/1",
            trainable=True,
        ),
        
        hub.text_embedding_column(
            key="summary_clean", 
            module_spec="https://tfhub.dev/google/Wiki-words-250-with-normalization/1",
            trainable=True,
        ),
        
    ]

In [24]:
# Create estimator train and evaluate function
def train_and_evaluate(output_dir, num_train_steps):
    estimator = tf.estimator.DNNClassifier(
        model_dir = output_dir,
        feature_columns = create_feature_cols(),
        hidden_units=[200, 100, 50],
        n_classes=len(assignee_vocabulary.tolist()),
        label_vocabulary = assignee_vocabulary.tolist(),
    )
    
#     estimator = tf.estimator.LinearClassifier(
#         model_dir = output_dir, 
#         feature_columns = create_feature_cols(),
#         n_classes=len(user_vocabulary.tolist()),
#         label_vocabulary = user_vocabulary.tolist(),
#     )
    train_spec = tf.estimator.TrainSpec(input_fn = make_input_fn(traindf, None),
                                        max_steps = num_train_steps)
    eval_spec = tf.estimator.EvalSpec(input_fn = make_input_fn(evaldf, 1), 
                                      steps = None, 
                                      start_delay_secs = 1, # start evaluating after N seconds, 
                                      throttle_secs = 10)  # evaluate every N seconds
    tf.estimator.train_and_evaluate(estimator, train_spec, eval_spec)

In [25]:
OUTDIR = './trained_model'

In [ ]:
# Run the model
shutil.rmtree(OUTDIR, ignore_errors = True)
train_and_evaluate(OUTDIR, 2000)

INFO:tensorflow:Using /tmp/tfhub_modules to cache modules.
INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_num_worker_replicas': 1, '_num_ps_replicas': 0, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f31146faef0>, '_keep_checkpoint_every_n_hours': 10000, '_tf_random_seed': None, '_keep_checkpoint_max': 5, '_log_step_count_steps': 100, '_save_checkpoints_secs': 600, '_model_dir': './trained_model', '_task_id': 0, '_service': None, '_device_fn': None, '_task_type': 'worker', '_master': '', '_train_distribute': None, '_evaluation_master': '', '_save_summary_steps': 100, '_is_chief': True, '_global_id_in_cluster': 0, '_save_checkpoints_steps': None, '_session_config': None}


/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


INFO:tensorflow:Running training and evaluation locally (non-distributed).
INFO:tensorflow:Start train and evaluate loop. The evaluate will happen after 10 secs (eval_spec.throttle_secs) or training is finished.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Initialize variable dnn/input_from_feature_columns/input_layer/description_clean_hub_module_embedding/module/embeddings/part_0:0,dnn/input_from_feature_columns/input_layer/description_clean_hub_module_embedding/module/embeddings/part_1:0 from checkpoint b'/tmp/tfhub_modules/0eaf8860f94909881c12ebab861d1ca7e8cd665b/variables/variables' with embeddings
INFO:tensorflow:Initialize variable dnn/input_from_feature_columns/input_layer/summary_clean_hub_module_embedding/module/embeddings/part_0:0,dnn/input_from_feature_columns/input_layer/summary_clean_hub_module_embedding/module/embeddings/part_1:0 from checkpoint b'/tmp/tfhub_modules/0eaf8860f94909881c12ebab861d1ca7e8cd665b/variables/variables' with embeddings
INFO:tensorflow:Done cal

INFO:tensorflow:Initialize variable dnn/input_from_feature_columns/input_layer/summary_clean_hub_module_embedding/module/embeddings/part_0:0,dnn/input_from_feature_columns/input_layer/summary_clean_hub_module_embedding/module/embeddings/part_1:0 from checkpoint b'/tmp/tfhub_modules/0eaf8860f94909881c12ebab861d1ca7e8cd665b/variables/variables' with embeddings
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from ./trained_model/model.ckpt-3
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 3 into ./trained_model/model.ckpt.
INFO:tensorflow:loss = 872.65875, step = 4
INFO:tensorflow:Saving checkpoints for 4 into ./trained_model/model.ckpt.
INFO:tensorflow:Loss for final step: 872.65875.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Initialize variable dnn/input_from_feature_columns/input_layer/description_c

INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from ./trained_model/model.ckpt-6
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 6 into ./trained_model/model.ckpt.
INFO:tensorflow:loss = 846.22974, step = 7
INFO:tensorflow:Saving checkpoints for 7 into ./trained_model/model.ckpt.
INFO:tensorflow:Loss for final step: 846.22974.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Initialize variable dnn/input_from_feature_columns/input_layer/description_clean_hub_module_embedding/module/embeddings/part_0:0,dnn/input_from_feature_columns/input_layer/description_clean_hub_module_embedding/module/embeddings/part_1:0 from checkpoint b'/tmp/tfhub_modules/0eaf8860f94909881c12ebab861d1ca7e8cd665b/variables/variables' with embeddings
INFO:tensorflow:Initialize variable dnn/input_from_feature_columns/input_layer/summa

...

INFO:tensorflow:loss = 268.6853, step = 896
INFO:tensorflow:Saving checkpoints for 896 into ./trained_model/model.ckpt.
INFO:tensorflow:Loss for final step: 268.6853.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Initialize variable dnn/input_from_feature_columns/input_layer/description_clean_hub_module_embedding/module/embeddings/part_0:0,dnn/input_from_feature_columns/input_layer/description_clean_hub_module_embedding/module/embeddings/part_1:0 from checkpoint b'/tmp/tfhub_modules/0eaf8860f94909881c12ebab861d1ca7e8cd665b/variables/variables' with embeddings
INFO:tensorflow:Initialize variable dnn/input_from_feature_columns/input_layer/summary_clean_hub_module_embedding/module/embeddings/part_0:0,dnn/input_from_feature_columns/input_layer/summary_clean_hub_module_embedding/module/embeddings/part_1:0 from checkpoint b'/tmp/tfhub_modules/0eaf8860f94909881c12ebab861d1ca7e8cd665b/variables/variables' with embeddings
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evalu

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



INFO:tensorflow:Loss for final step: 284.28558.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Initialize variable dnn/input_from_feature_columns/input_layer/description_clean_hub_module_embedding/module/embeddings/part_0:0,dnn/input_from_feature_columns/input_layer/description_clean_hub_module_embedding/module/embeddings/part_1:0 from checkpoint b'/tmp/tfhub_modules/0eaf8860f94909881c12ebab861d1ca7e8cd665b/variables/variables' with embeddings
INFO:tensorflow:Initialize variable dnn/input_from_feature_columns/input_layer/summary_clean_hub_module_embedding/module/embeddings/part_0:0,dnn/input_from_feature_columns/input_layer/summary_clean_hub_module_embedding/module/embeddings/part_1:0 from checkpoint b'/tmp/tfhub_modules/0eaf8860f94909881c12ebab861d1ca7e8cd665b/variables/variables' with embeddings
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2018-07-27-17:44:30
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from ./trained_m

INFO:tensorflow:Initialize variable dnn/input_from_feature_columns/input_layer/summary_clean_hub_module_embedding/module/embeddings/part_0:0,dnn/input_from_feature_columns/input_layer/summary_clean_hub_module_embedding/module/embeddings/part_1:0 from checkpoint b'/tmp/tfhub_modules/0eaf8860f94909881c12ebab861d1ca7e8cd665b/variables/variables' with embeddings
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2018-07-27-17:48:07
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from ./trained_model/model.ckpt-938
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2018-07-27-17:48:10
INFO:tensorflow:Saving dict for global step 938: accuracy = 0.33131948, average_loss = 3.1789126, global_step = 938, loss = 404.74896
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 938: ./trained_model/model.ckpt-938
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Init

INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2018-07-27-17:51:42
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from ./trained_model/model.ckpt-941
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2018-07-27-17:51:45
INFO:tensorflow:Saving dict for global step 941: accuracy = 0.33289027, average_loss = 3.1868446, global_step = 941, loss = 405.75885
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 941: ./trained_model/model.ckpt-941
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Initialize variable dnn/input_from_feature_columns/input_layer/description_clean_hub_module_embedding/module/embeddings/part_0:0,dnn/input_from_feature_columns/input_layer/description_clean_hub_module_embedding/module/embeddings/part_1:0 from checkpoint b'/tmp/tfhub_modules/0eaf8860f94909881c12ebab861d1ca7e8cd665b/variables/variables' with embeddings
INFO:tensorf